In [1]:
import pickle
import random as rand
import random
import gym
import pandas as pd
import math
import numpy as np

In [ ]:
runOnly = False

In [22]:
num_configs = math.factorial(9)
GOAL_STATE_VAL = (0,1,2,3,4,5,6,7,8)
GOAL_STATE = 322560 
trans_matrix = np.load("trans_matrix.npy")
states_df = pd.read_csv("states.csv")   
qtable = np.load('qtable2.npy')

In [3]:
def strToArray(s):
    return [int(a) for a in s[1:-1].split(',')]

In [4]:
states_val_dict = {}
states_num_dict = {}
for i in range(len(states_df)):
    row = states_df.iloc[i]
    rowv = strToArray(row.value)
    states_val_dict[tuple(rowv)] = i
    states_num_dict[i] = tuple(rowv)
    pass
    

In [23]:
class SlidingPuzzleEngine:
    def __init__(self, states_val_dict, states_num_dict):
        self.TOPS = [0,1,2]
        self.RIGHTS = [2,5, 8]
        self.LEFTS = [0, 3, 6]
        self.BOTTOMS = [6, 7, 8]
        self.moves = ["Up", "Down", "Left", "Right"]
        self.states_val_dict = states_val_dict
        self.states_num_dict = states_num_dict
    
    def _swap(self, arr, in1, in2):
        temp = arr[in2]
        arr[in2] = arr[in1]
        arr[in1] = temp
         
    
    def move(self,stateNum, m):
        stateValue = (list(self.states_num_dict[stateNum]))
        nextStateValue = stateValue
        
        if m == 1 or m == "Up":
            if not stateValue.index(0) in self.TOPS:
                self._swap(stateValue, stateValue.index(0), stateValue.index(0)-3) 
            else:
                nextStateValue = stateValue
                
            
        elif m == 2 or m == "Down":
            if not stateValue.index(0) in self.BOTTOMS:
                self._swap(stateValue, stateValue.index(0), stateValue.index(0)+3) 
            else:
                nextStateValue = stateValue 
        
        elif m == 3 or m == "Left":
            if not stateValue.index(0) in self.LEFTS:
                self._swap(stateValue, stateValue.index(0), stateValue.index(0)-1) 
            else:
                nextStateValue = stateValue
            pass
        
        else:
            if not stateValue.index(0) in self.RIGHTS:
                self._swap(stateValue, stateValue.index(0), stateValue.index(0)+1) 
            else:
                nextStateValue = stateValue
            pass
        
        nextState = self.states_val_dict[(tuple(nextStateValue))]
        return (nextStateValue), nextState
    
    def display(self, stateNum):
        stateVal = (self.states_num_dict[stateNum] )
        print(stateVal)
        
        print("-------------")
        print("|" ,stateVal[0] , "|", stateVal[1], "|", stateVal[2], "|")
        print("-------------")
        print("|" ,stateVal[3] , "|", stateVal[4], "|", stateVal[5], "|")
        print("-------------")
        print("|" ,stateVal[6] , "|", stateVal[7], "|", stateVal[8], "|") 
        print("-------------")
        
    def distance(self, stateValue):
        goalState = [0,1,2,3,4,5,6,7,8]
        return sum([abs(stateValue.index(a) - a) for a in stateValue ])
    
    def stateScore(self, stateNum):
        stateValue = self.states_num_dict[stateNum]
        return sum([ (stateValue[i])**(1+i) for i in range(9)])
    
    
    def getAllowedActions(self, state):
        allowedActions = np.asarray( trans_matrix[state, 1:5] != state)*[1,2,3,4]
        allowedActions = allowedActions[allowedActions >0]-1 
        
        return allowedActions
        
    
    def shuffle(self):
        stateVal = [0,1,2,3,4,5,6,7,8]
        state = GOAL_STATE  
        moves = rand.randint(1,70) 
        for i in range(moves):
            stateVal, state = self.move(state, self.moves[rand.sample(list(self.getAllowedActions(state)), 1)[0]] )
            
            
        return stateVal,  state
    


In [24]:
sEngine = SlidingPuzzleEngine( states_val_dict, states_num_dict)

In [25]:
s = sEngine

In [26]:

if runOnly:
    for i in range(num_configs):
        if i == GOAL_STATE or True:
            _, trans_matrix[i, 1] = s.move(trans_matrix[i, 0], "Up")
            _, trans_matrix[i, 2] = s.move(trans_matrix[i, 0], "Down")
            _, trans_matrix[i, 3] = s.move(trans_matrix[i, 0], "Left")
            _, trans_matrix[i,4] = s.move(trans_matrix[i, 0], "Right")

In [28]:
np.save("trans_matrix.npy", trans_matrix)

In [48]:
class SlidingPuzzleEnv(gym.Env):
    
    def __init__(self, engine):
        self.actions = ["Up","Down","Left", "Right"]
        self.action_space = gym.spaces.Discrete(4)
        self.observation_space = gym.spaces.Discrete(9)
        self.engine = engine
        self.stateVal,_ = self.engine.shuffle()
        self.state = states_val_dict[tuple(self.stateVal)]
        self.moves = 0
        
    def step(self,action, max_steps):
        self.moves+=1
        prevState = self.state
        prevStateScore = trans_matrix[prevState, 5]
        self.stateVal, self.state = self.engine.move(self.state,action)  
        stateScore = trans_matrix[self.state, 5]
        reward = stateScore - prevStateScore - self.moves
        done = self.state == GOAL_STATE 
        if done:
            reward += 1000000000
        elif self.moves == max_steps:
            reward += -10000000000
            
        
        return self.state, reward, done , {}
    def getAllowedActions(self, state):
#         allowedActions = np.asarray( trans_matrix[state, 1:5] != state)
        
        return self.engine.getAllowedActions(state)
        
    
    
    def reset(self):
        self.stateVal, self.state = self.engine.shuffle() 
        return self.state 
    
    def given(self):
        self.state = states_val_dict[(7,2,4,5,0,6,8,3,1)]
        return self.state
    
    def getBestAction(self, state):
        allowedActions = env.getAllowedActions(state)

    #     actScores = qtable[state, allowedActions]
        bestActionIndex = 0
        bestScore = -math.inf
        for al in allowedActions:
            score = qtable[state,al]
#             print(score)
            if score > bestScore:
                bestScore = score
                bestActionIndex = al
        return bestActionIndex
    
    def getBestScore(self, state):
        allowedActions = env.getAllowedActions(state)
 
        bestActionIndex = 0
        bestScore = -math.inf
        for al in allowedActions:
            score = qtable[state,al] 
            if score > bestScore:
                bestScore = score
                bestActionIndex = al
        return bestScore
    
    def getRandomAction(self, state):
        
        allowedActions = env.getAllowedActions(state)

        return rand.sample(list(allowedActions), 1)[0]
        
    
    
    
    

        
    

In [49]:
env = SlidingPuzzleEnv(sEngine)

In [55]:

if runOnly:
    qtable = np.zeros((num_configs, 4)) 
else:
    qtable = np.load('qtable2.npy')
    
total_episodes = 10000       # Total episodes
learning_rate = 0.01          # Learning rate
max_steps = 50               # Max steps per episode
gamma = 0.95                 # Discounting rate

# Exploration parameters

rewards = []
episode = 0
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.05           # Minimum exploration probability 
decay_rate = 0.0001          # Exponential decay rate for exploration prob

In [57]:
# %%time
# learning_rate = 0.01
# # epsilon = 1
# min_epsilon = 0.1
# total_episodes = 10000000
# max_steps = 50
# 2 For life or until learning is stopped
# max_steps = 100
# epsilon  = max_epsilon
# decay_rate = 
rewards = []
epsilon = 1.0
min_epsilon = 0.15
while True:
    # Reset the environment
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0
    
    for step in range(max_steps):
        exp_exp_tradeoff = random.uniform(0, 1)
        
        if exp_exp_tradeoff > epsilon: 
            action = env.getBestAction(state)

        else:
            action = env.getRandomAction(state)
#             a = np.array([1,2,3,4])*env.getAllowedActions(state)
#             a = a[a> 0]
#             action = rand.sample(list(a)  , 1)[0]-1
            
        
        new_state, reward, done, info = env.step(env.actions[action], max_steps)
        qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * env.getBestScore(new_state) - qtable[state, action])
        
        total_rewards += reward
        
        state = new_state
        
        if done == True: 
            break
        
    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode) 
    rewards.append(total_rewards)
    episode +=1
    


print ("Score over time: " +  str(sum(rewards)/total_episodes))
print(qtable)

KeyboardInterrupt: 

In [58]:
episode, decay_rate, min_epsilon, max_epsilon, np.exp(-decay_rate*episode)

(2656958, 0.0001, 0.15, 1.0, 4.0717414690320313e-116)

In [67]:
np.save(  "qtable2.npy", qtable)

In [65]:

for episode in range(5):
    state = env.reset()
    step = 0
    done = False
    print("****************************************************")
    print("EPISODE ", episode)

    for step in range(max_steps):
         
        action = env.getBestAction(state)
        
        new_state, reward, done, info = env.step(env.actions[action], max_steps)
        if step == 0 or step == max_steps-1 or True  :
            sEngine.display( state )
            print("step:", step)
        if done: 
            if new_state == GOAL_STATE:
                print("We reached our Goal 🏆")
            else:
                print("You failed ☠️")
             
            print("Number of steps", step)
            
            break
        state = new_state 

****************************************************
EPISODE  0
(1, 2, 7, 4, 6, 5, 3, 0, 8)
-------------
| 1 | 2 | 7 |
-------------
| 4 | 6 | 5 |
-------------
| 3 | 0 | 8 |
-------------
step: 0
(1, 2, 7, 4, 0, 5, 3, 6, 8)
-------------
| 1 | 2 | 7 |
-------------
| 4 | 0 | 5 |
-------------
| 3 | 6 | 8 |
-------------
step: 1
(1, 2, 7, 0, 4, 5, 3, 6, 8)
-------------
| 1 | 2 | 7 |
-------------
| 0 | 4 | 5 |
-------------
| 3 | 6 | 8 |
-------------
step: 2
(1, 2, 7, 3, 4, 5, 0, 6, 8)
-------------
| 1 | 2 | 7 |
-------------
| 3 | 4 | 5 |
-------------
| 0 | 6 | 8 |
-------------
step: 3
(1, 2, 7, 3, 4, 5, 6, 0, 8)
-------------
| 1 | 2 | 7 |
-------------
| 3 | 4 | 5 |
-------------
| 6 | 0 | 8 |
-------------
step: 4
(1, 2, 7, 3, 4, 5, 6, 8, 0)
-------------
| 1 | 2 | 7 |
-------------
| 3 | 4 | 5 |
-------------
| 6 | 8 | 0 |
-------------
step: 5
(1, 2, 7, 3, 4, 0, 6, 8, 5)
-------------
| 1 | 2 | 7 |
-------------
| 3 | 4 | 0 |
-------------
| 6 | 8 | 5 |
-------------
step: 

In [66]:


for episode in range(1):
    state = env.given()
    step = 0
    done = False
    print("****************************************************")
    print("EPISODE ", episode)

    for step in range(max_steps): 
        action = env.getBestAction(state)
        
        new_state, reward, done, info = env.step(env.actions[action], max_steps)
        if step == 0 or step == max_steps-1 or True :
            sEngine.display( state )
            print("step:", step) 
        if done:
             
            if new_state == GOAL_STATE:
                print("We reached our Goal 🏆")
            else:
                print("You failed ☠️")
             
            print("Number of steps", step)
            
            break
        state = new_state 

****************************************************
EPISODE  0
(7, 2, 4, 5, 0, 6, 8, 3, 1)
-------------
| 7 | 2 | 4 |
-------------
| 5 | 0 | 6 |
-------------
| 8 | 3 | 1 |
-------------
step: 0
(7, 2, 4, 5, 3, 6, 8, 0, 1)
-------------
| 7 | 2 | 4 |
-------------
| 5 | 3 | 6 |
-------------
| 8 | 0 | 1 |
-------------
step: 1
(7, 2, 4, 5, 3, 6, 0, 8, 1)
-------------
| 7 | 2 | 4 |
-------------
| 5 | 3 | 6 |
-------------
| 0 | 8 | 1 |
-------------
step: 2
(7, 2, 4, 0, 3, 6, 5, 8, 1)
-------------
| 7 | 2 | 4 |
-------------
| 0 | 3 | 6 |
-------------
| 5 | 8 | 1 |
-------------
step: 3
(7, 2, 4, 3, 0, 6, 5, 8, 1)
-------------
| 7 | 2 | 4 |
-------------
| 3 | 0 | 6 |
-------------
| 5 | 8 | 1 |
-------------
step: 4
(7, 2, 4, 3, 6, 0, 5, 8, 1)
-------------
| 7 | 2 | 4 |
-------------
| 3 | 6 | 0 |
-------------
| 5 | 8 | 1 |
-------------
step: 5
(7, 2, 4, 3, 6, 1, 5, 8, 0)
-------------
| 7 | 2 | 4 |
-------------
| 3 | 6 | 1 |
-------------
| 5 | 8 | 0 |
-------------
step: 